# Projectile Motion
## Differential Equations and Optimization

A baseball player hits a ball (with a mass of 0.145 kg) that is initially one meter above the ground.  The ball leaves the bat with a speed of 45 m/s at an angle of 30 degrees relative to the horizontal.   Assuming no air resistance, 

1. What is the horizontal distance the ball travels before hitting the ground?
2. If the batter has control of the initial angle, what is the optimal angle to maximize the horizontal distance before the ball hits the ground?

Recalculate the above assuming air drag of the form $F_\mbox{drag} = -c V|V|$ where $c = 0.001$ in SI units.


### Procedure - No air resistance

1. Set up a system of differential equations with event detection enabled.
2. Solve the system numerically

3. Write a function to calculate the distance traveled as a function of angle.
4. Use Scipy’s optimization package to maximize the distance

Since this part can be done analytically, we will use sympy to check our work.

### Procedure - With air resistance

Same as steps 1-4 above.  This time there is no analytical solution.



In [1]:
import numpy as np

from scipy.integrate import solve_ivp
from scipy.optimize import minimize

from sympy import *
from IPython.display import display, Latex

import matplotlib.pyplot as plt

%matplotlib inline

### Equations of Motion
\begin{array}{cll}
\frac{dx}{dt} & = & V_x \\
m\frac{dV_x}{dt} & = & 0 \\
\frac{dy}{dt} & = & V_y \\
m\frac{dV_y}{dt} & = & -mg
\end{array}

Initial Conditions:
\begin{array}{lll}
x(0) & = & 0 \\
V_x(0) & = & V\cos(\theta) \\
y(0) & = & 1\mbox{m} \\
V_y(0) & = & V\sin(\theta)
\end{array}

<img src="initial_velocity.png" height="60%"  width="60%">


In [2]:
#  Define our susytem of ODEs.  Must have the form df/dt = f(t,X)
def no_drag(t, X):
    
    g = 9.8
    
    Xprime = np.zeros(4)
    
    #  Vector format
    #   x[0] = x position
    #   x[1] = x velocity
    #   x[2] = y position
    #   x[3] = y velocity
    
    Xprime[0] = X[1]
    Xprime[1] = 0
    Xprime[2] = X[3]
    Xprime[3] = -g
    
    return Xprime

#  Function for event detection.  Terminate integration when y crosses zero moving from positive to negative.  y is in the variable X[2]
def events(t, X):
    return X[2]

events.terminal = True
events.direction = -1

# Function to solve our differential equations and return the value for the total x distance traveled.
def calculate_range_no_drag(theta):
    #  Angle needs to be in radians
    theta = np.radians(theta[0])
    
    #  Initial velcoity in m/s
    V0 = 45
    
    #  We use a while loop breaking out of it if we detect an event
    #  These kkep track of total iterations and provides us a means to break out if max counts is exceeded.
    count = 1
    max_count = 100
    
    #  Define the number of points to be returned and the iintegration time
    N = 1000
    tf = 20
    
    while count <= max_count:
        count = count + 1
    
        #  Define time spane and points to return
        tspan = (0, tf)
        t_eval = np.linspace(tspan[0], tspan[1], N)
    
        #  Initial conditions
        X0 = np.array( [0, V0 * np.cos(theta), 1, V0 * sin(theta)])

        #  Solve the system
        sol = solve_ivp(no_drag, tspan, X0, t_eval = t_eval, events = events)
        
        #  Check to make sure event is detected.  If not, restart with a larger integration time
        if sol.t_events[0].size > 0:
            R = sol.y[0, -1]
            #print('Event exists ', sol.t_events)
            break;
        else:
            #print('No events detected.')
            tf = tf + 0.1 * tf
            N = N + 0.1 * N
            R = []
            
            
    return R

#  Scipy's routines find minimums, not maximums.  Our objective function takes the negative value so we can minimze the function
def objective(theta):
    return -calculate_range_no_drag(theta)

In [3]:
#  Answer to 1.1
print( calculate_range_no_drag((30,)))

bounds = ( (0.1, 89.9), )
results = minimize(objective, (25,), bounds = bounds, method = 'Nelder-Mead')
print(results)

180.22690835513987
 final_simplex: (array([[44.83528137],
       [44.83530045]]), array([-207.63011637, -207.63004764]))
           fun: -207.63011636557692
       message: 'Optimization terminated successfully.'
          nfev: 61
           nit: 30
        status: 0
       success: True
             x: array([44.83528137])


C:\Users\moone\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:516: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  RuntimeWarning)


The differential equations above can be solved analytically.  If the ball travels horizontally a distance $R$ in time, $t$, we can write,
\begin{array}{llll}
x(t) & = & R = & V_0\cos(\theta)t  \\
y(t) & = & 0 = & y_0 + V_0\sin(\theta)t - \frac{1}{2}gt^2
\end{array}
We can solve the top equation for $t$ and plug it into the equation for $y$.  This gives,
$$
y_0 + R \tan(\theta) - \frac{1}{2}\frac{g}{V_0^2\cos^2(\theta)}R^2 = 0.
$$
We have a quadratic equation in $R$.  Once we solve for $R$, we can differentiate with respect to $\theta$ to find the angle which maximizes $R$.

In [4]:
#  Define constant for convenience
a = -9.8 / 2 / 45**2

#  Define our syumbols  and expression
y0, A, theta, R = symbols('y0, A, theta, R')
expr = y0 + + tan(theta) * R + A * R**2 / cos(theta)**2

#  Solve for range
R = solve(expr, R)

#  There are two answers.  Our happens to be in the second entry
R = R[1]
display(R)

#  Subsistute in our values and compare with numerical answer
print( R.subs( [(y0, 1), (A, a), (theta, np.radians(30))]) )
print( calculate_range_no_drag( (30,)) )

-(sqrt(-4*A*y0 + sin(theta)**2) + sin(theta))*cos(theta)/(2*A)

180.664729951254
180.22690835513987


In [5]:
#  Calculate the derivative to find the max
dR = diff(R, theta)
display(dR)
dR = dR.subs( [(y0, 1), (A, a)])

#  Solve for max and print the result
ans = solve(dR, theta)
ans = ans[0]
print(ans * 180 / np.pi)

(sqrt(-4*A*y0 + sin(theta)**2) + sin(theta))*sin(theta)/(2*A) - (cos(theta) + sin(theta)*cos(theta)/sqrt(-4*A*y0 + sin(theta)**2))*cos(theta)/(2*A)

44.8620255528842


In [6]:
#  Clear out symbolic variables just so there are no potential conflicts
del(y0); del(A); del(theta); del(R); del(expr)

## Part 2
Include the drag force given by,
$$
F_{\mbox{drag}} = -cv|v|,
$$
where $c$ = 0.001.
The equations of motion are now,
\begin{array}{cll}
\frac{dx}{dt} & = & V_x \\
m\frac{dV_x}{dt} & = & -cv_x|v_x| \\
\frac{dy}{dt} & = & V_y \\
m\frac{dV_y}{dt} & = & -mg - cv_y|v_y|
\end{array}.
The initial conditions are the same as inthe previous part.


In [7]:
def drag(t, X):
    Xprime = np.zeros(4)
    
    #  The mass is needed for the nonlinear p[roblem]
    m = 0.145
    c = 0.001
    
    #  For simplicity, we define the ddrag forces here.
    F_x = -c * X[1] * np.abs(X[1])
    F_y = -c * X[3] * np.abs(X[3])
    
    Xprime[0] = X[1]
    Xprime[1] = F_x / m
    Xprime[2] = X[3]
    Xprime[3] = -9.8 + F_y / m
        
    return Xprime

#  Range function for the drag case
def calculate_range_drag(theta):
    theta = np.radians(theta[0])
    
    V0 = 45
    
    count = 1
    max_count = 100
    
    N = 1000
    tf = 5
    
    while count <= max_count:
        count = count + 1
    
        tspan = (0, tf)
        t_eval = np.linspace(tspan[0], tspan[1], N)
    
        X0 = np.array( [0, V0 * np.cos(theta), 1, V0 * sin(theta)])

        sol = solve_ivp(drag, tspan, X0, t_eval = t_eval, events = events)
        if sol.t_events[0].size > 0:
            R = sol.y[0, -1]
            #print('Event exists ', sol.t_events)
            break;
        else:
            #print('No events detected.')
            tf = tf + 0.1 * tf
            N = N + 0.1 * N
            
        
    
    return R

def objective_drag(theta):
    return -calculate_range_drag(theta)

In [8]:
#  Optimize angle
results = minimize(objective_drag, (25,), bounds = bounds, method = 'Nelder-Mead')
print(results)

 final_simplex: (array([[41.24008179],
       [41.24015808]]), array([-118.02300058, -118.02290623]))
           fun: -118.02300058102726
       message: 'Optimization terminated successfully.'
          nfev: 50
           nit: 23
        status: 0
       success: True
             x: array([41.24008179])
